In [1]:
from elasticsearch import Elasticsearch


# url ="https://booksearch.es.us-central1.gcp.cloud.es.io/"
# cloud_id = "booksearch:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDg0NjljZWUyZGI2MjRkNmZiYzA0ZjM2NjJmYjgxMjIyJGMyN2Y2YzE4OGYxMzQ5ZDFhMzRmZDE4NjY3MjU0OWIy"
username = "elastic"
password = "CEXkB1JXAzS1kXcjVnQJhX0k"
# Tạo đối tượng Elasticsearch

es = Elasticsearch(
    cloud_id="ba33129799b54d9394d6041531f220b4:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQyY2UxMjA3MTgyMGQ0NmE4OWEyNmY0YzI0NTRkNDlkMyQ1N2I2ZTc5Mjc3OGM0Y2EyYjlhYTMwMzljNzMzMzU2Zg==",
    basic_auth=(username, password)
)
print(es.ping)



<bound method Elasticsearch.ping of <Elasticsearch(['https://2ce12071820d46a89a26f4c2454d49d3.us-central1.gcp.cloud.es.io:443'])>>


In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')
from tqdm.auto import tqdm
tqdm.pandas()
import uuid
from elasticsearch import helpers

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class Tokenizer(object):
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

    def get_token(self, documents):
        sentences  = [documents]
        sentence_embeddings = self.model.encode(sentences)
        encod_np_array = np.array(sentence_embeddings)
        encod_list = encod_np_array.tolist()
        return encod_list[0]

In [4]:
token_instance = Tokenizer()

In [15]:
#Read dataset and cleaning data
import pandas as pd

# Load the dataset
df = pd.read_csv("GoogleBookAPIDataset.csv")

# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0'])

# Handle duplicates
df = df.drop_duplicates()

# Standardize column names
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Handle missing values
df = df.dropna()

df = df.iloc[:, 1:]

print(df.columns)





Index(['id', 'title', 'desc', 'authors', 'categories', 'averagerating',
       'maturityrating', 'publisheddate', 'pagecount'],
      dtype='object')


In [5]:
import pandas as pd

# Load the dataset
df_vn = pd.read_csv("books.csv")

# Drop unnecessary columns
# df_vn = df_vn.drop(columns=['Unnamed: 0'])

# Handle duplicates
df_vn = df_vn.drop_duplicates()

# Replace blank values with null values
df_vn = df_vn.fillna("null")

# Standardize column names
df_vn.columns = df_vn.columns.str.lower().str.replace(' ', '_')

df_vn['averagerating'] = pd.to_numeric(df_vn['averagerating'], errors='coerce').fillna(0)
df_vn['publisheddate'] = pd.to_datetime(df_vn['publisheddate'], errors='coerce').fillna(pd.to_datetime('1970-01-01'))
df_vn['pagecount'] = pd.to_numeric(df_vn['pagecount'], errors='coerce').fillna(0)



# Handle missing values
df_vn = df_vn.dropna()

print(df_vn.columns)


print(df_vn[:6])

Index(['id', 'title', 'desc', 'authors', 'categories', 'averagerating',
       'maturityrating', 'publisheddate', 'pagecount'],
      dtype='object')
             id                                              title  \
0  TO_dEAAAQBAJ                               Lập trình với Python   
1  evDdEAAAQBAJ                     Coding 10 Lập trình với Python   
2  NF3oEAAAQBAJ                      Coding 8 Lập trình với Python   
3  O99WEAAAQBAJ  Sách luyện thi Hội thi Tin học trẻ với Python ...   
4  w-TaAAAAMAAJ    Động vật chí Việt Nam: Phân bộ̣ rá̆n, Serpentes   
5  ExKs2q51U9UC                         Tập san nghiên cứu lịch sử   

                                                desc   authors categories  \
0  Python là gì Theo chỉ số phổ biến của ngôn ngữ...  VIETSTEM  Education   
1  Tại sao cần yêu công nghệ, cần làm chủ công ng...  VIETSTEM  Education   
2                                               null  VIETSTEM  Computers   
3  Trong năm 2021, với cú pháp đơn giản, dễ học, ..

In [17]:
# concat 2 data

result = pd.concat([df, df_vn], axis=0)

print(result[:5])

# result.to_csv('data_books_test.csv', index=False, encoding="utf-8")

# result.drop_duplicates()


# result = pd.read_csv('total_books.csv')
# result = result.fillna("null")
# result


             id                                              title  \
0  L2EQuwEACAAJ             Harry Potter and the Half-Blood Prince   
1  YlMOvgAACAAJ             Harry Potter and the Half-blood Prince   
2  8V9nnAEACAAJ             Harry Potter and the Half-Blood Prince   
3  1YwtPwAACAAJ             Harry Potter and the Half-Blood Prince   
4  ONbSoEKGthgC  ハリー・ポッターと謎のプリンス - Harry Potter and the Half-Bl...   

                                                desc            authors  \
0  As Harry enters his sixth year at Hogwarts, a ...  ['J. K. Rowling']   
1  Paperback edition of the sixth book in J.K. Ro...  ['J. K. Rowling']   
2  Sequel to: Harry Potter and the Order of the P...  ['J. K. Rowling']   
3  Sixth-year Hogwarts student Harry Potter gains...  ['J. K. Rowling']   
4  ヴォルデモートの復活のせいで、夏だというのに国中に冷たい霧が立ち込めていた。そんな中を、ダー...   ['J.K. Rowling']   

               categories  averagerating maturityrating publisheddate  \
0             ['England']            4.5     NOT_MATURE

In [18]:
row_with_id = result.loc[result['id'] == 'TO_dEAAAQBAJ']

# In ra dòng có id 'TO_dEAAAQBAJ'
print(row_with_id)

             id                 title  \
0  TO_dEAAAQBAJ  Lập trình với Python   

                                                desc   authors categories  \
0  Python là gì Theo chỉ số phổ biến của ngôn ngữ...  VIETSTEM  Education   

   averagerating maturityrating        publisheddate  pagecount  
0            0.0     NOT_MATURE  2023-10-19 00:00:00      234.0  


In [19]:
# delete index
es.indices.delete(index="books", ignore=[400, 404])

C:\Users\DELL\AppData\Local\Temp\ipykernel_3368\525162218.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="books", ignore=[400, 404])


ObjectApiResponse({'error': {'root_cause': [{'type': 'index_not_found_exception', 'reason': 'no such index [books]', 'resource.type': 'index_or_alias', 'resource.id': 'books', 'index_uuid': '_na_', 'index': 'books'}], 'type': 'index_not_found_exception', 'reason': 'no such index [books]', 'resource.type': 'index_or_alias', 'resource.id': 'books', 'index_uuid': '_na_', 'index': 'books'}, 'status': 404})

In [10]:
# convert description to vector of English Books
df['vector'] = df['desc'].progress_apply(token_instance.get_token)

100%|██████████| 22332/22332 [29:20<00:00, 12.68it/s] 


In [6]:
# convert description to vector of VN Books
df_vn['vector'] = df_vn['desc'].progress_apply(token_instance.get_token)
print(df_vn.columns)


100%|██████████| 2321/2321 [02:37<00:00, 14.78it/s]

Index(['id', 'title', 'desc', 'authors', 'categories', 'averagerating',
       'maturityrating', 'publisheddate', 'pagecount', 'vector'],
      dtype='object')


In [20]:
result['vector'] = result['desc'].progress_apply(token_instance.get_token)
print(result.columns)

100%|██████████| 24653/24653 [29:56<00:00, 13.72it/s] 

Index(['id', 'title', 'desc', 'authors', 'categories', 'averagerating',
       'maturityrating', 'publisheddate', 'pagecount', 'vector'],
      dtype='object')


In [27]:
print(result.columns)
print(df.columns)

Index(['id', 'title', 'desc', 'authors', 'categories', 'averagerating',
       'maturityrating', 'publisheddate', 'pagecount'],
      dtype='object')
Index(['id', 'title', 'desc', 'authors', 'categories', 'averagerating',
       'maturityrating', 'publisheddate', 'pagecount'],
      dtype='object')


In [21]:
#original
#mapping step



mappings = {
    #tokenizing of data to words
    "properties":{
        "id":{"type":"keyword"},
        "title":{"type":"text", "analyzer":"english"},
        "desc":{"type":"text"},
        "desc_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "l2_norm"},
        "authors":{"type":"keyword"},
        "categories": {"type": "keyword"},
        "averageRating": {"type": "float"},
        "maturityRating": {"type": "keyword"},
        "publishedDate": {"type": "date"},
        "pageCount": {"type": "integer"}
    }
}

# create index books
es.indices.create(index="books", mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'books'})

In [7]:
#test

mappings = {
    "properties": {
        "id": {"type": "keyword"},
        "title": {
            "type": "text",
            "analyzer": "english",
            "fields": {
                "autocomplete": {
                    "type": "completion"
                }
            }
        },
        "desc": {"type": "text"},
        "desc_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "l2_norm"},
        "authors": {"type": "keyword"},
        "categories": {"type": "keyword"},
        "averageRating": {"type": "float"},
        "maturityRating": {"type": "keyword"},
        "publishedDate": {"type": "date"},
        "pageCount": {"type": "integer"}
    }
}

# create index books
es.indices.create(index="books_test", mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'books_test'})

In [36]:
# indexed documents
for i, row in df.iterrows():
    doc = {
        "id": row["id"],
        "title": row["title"],
        "desc": row["desc"],
        "authors": row["authors"],
        "categories": row["categories"],
        "averagerating": row["averagerating"],
        "maturityrating": row["maturityrating"],
        "publisheddate": row["publisheddate"],
        "pagecount": row["pagecount"]
    }
            
    es.index(index="books", id=i, document=doc)

KeyboardInterrupt: 

In [12]:
# another way to store data to index
from elasticsearch.helpers import bulk

bulk_data = []
for i,row in df.iterrows():
    bulk_data.append(
        {
            "_index": "books",
            "_id": row["id"],
            "_source": {        
                "id": row["id"],
        "title": row["title"],
        "desc": row["desc"],
        "desc_vector": row["vector"],
        
        "authors": row["authors"],
        "categories": row["categories"],
        "averagerating": row["averagerating"],
        "maturityrating": row["maturityrating"],
        "publisheddate": row["publisheddate"],
        "pagecount": row["pagecount"],
            }
        }
    )
bulk(es, bulk_data)

(22332, [])

In [12]:
# df_vn.to_csv('check_vn_book.csv', index = False, encoding="utf-8")





In [13]:
# another way to store data to index
from elasticsearch.helpers import bulk

bulk_data = []
for i,row in df_vn.iterrows():
    bulk_data.append(
        {
            "_index": "books",
            "_id": row["id"],
            "_source": {        
                "id": row["id"],
        "title": row["title"],
        "desc": row["desc"],
        "desc_vector": row["vector"],
        "authors": row["authors"],
        "categories": row["categories"],
        "averagerating": row["averagerating"],
        "maturityrating": row["maturityrating"],
        "publisheddate": row["publisheddate"],
        "pagecount": row["pagecount"],
            }
        }
    )
bulk(es, bulk_data)

(2321, [])

In [8]:
#test autocomplete
# another way to store data to index
from elasticsearch.helpers import bulk

bulk_data = []
for i,row in df_vn.iterrows():
    bulk_data.append(
        {
            "_index": "books_test",
            "_id": row["id"],
            "_source": {        
                "id": row["id"],
        "title": row["title"],
        "desc": row["desc"],
        "desc_vector": row["vector"],
        "authors": row["authors"],
        "categories": row["categories"],
        "averagerating": row["averagerating"],
        "maturityrating": row["maturityrating"],
        "publisheddate": row["publisheddate"],
        "pagecount": row["pagecount"],
            }
        }
    )
bulk(es, bulk_data)

(2321, [])

In [22]:
# another way to store data to index
from elasticsearch.helpers import bulk

bulk_data = []
for i,row in result.iterrows():
    bulk_data.append(
        {
            "_index": "books",
            "_id": row["id"],
            "_source": {        
                "id": row["id"],
        "title": row["title"],
        "desc": row["desc"],
        "desc_vector": row["vector"],
        
        "authors": row["authors"],
        "categories": row["categories"],
        "averagerating": row["averagerating"],
        "maturityrating": row["maturityrating"],
        "publisheddate": row["publisheddate"],
        "pagecount": row["pagecount"],
            }
        }
    )
bulk(es, bulk_data)



(24653, [])

In [23]:
# ounting the number of items in the index:
es.indices.refresh(index="books")
es.cat.count(index="books", format="json")

ListApiResponse([{'epoch': '1708595720', 'timestamp': '09:55:20', 'count': '24617'}])

In [16]:
INPUT = input("Enter the Input Query ")
token_vector = token_instance.get_token(INPUT)
print(token_vector)
res = es.search(
    index="books",
    knn  ={
                "field": "desc_vector",
                "query_vector": token_vector,
                "k": 10,
                "num_candidates": 50
   
            }
,
    request_timeout=55
)

title = [x['_source']['title']  for x in res['hits']['hits']] 
title


[-0.06277179718017578, 0.05495882406830788, 0.052164819091558456, 0.08579002320766449, -0.08274894207715988, -0.07457292824983597, 0.06855469942092896, 0.01839641109108925, -0.08201132714748383, -0.03738480061292648, 0.012124930508434772, 0.0035183443687856197, -0.004134284798055887, -0.043784432113170624, 0.021807366982102394, -0.005102717317640781, 0.019546568393707275, -0.04234876111149788, -0.11035963147878647, 0.005424485541880131, -0.05573485791683197, 0.028052441775798798, -0.023158691823482513, 0.02848139964044094, -0.05370962619781494, -0.052601657807826996, 0.03393928334116936, 0.0453886054456234, 0.023718446493148804, -0.0731208473443985, 0.05477773770689964, 0.017047306522727013, 0.08136031031608582, -0.002862680936232209, 0.011958093382418156, 0.07355853170156479, -0.09423752874135971, -0.08136200904846191, 0.04001541808247566, 0.0006921236636117101, -0.0133932800963521, -0.05453804135322571, 0.005151398945599794, -0.026139816269278526, 0.03680703043937683, -0.033959660679

C:\Users\DELL\AppData\Local\Temp\ipykernel_19824\2905709573.py:4: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  res = es.search(


['The Fairy Godmother',
 'Getting Rid Of Bradley',
 'Suffer the Children',
 'Captivating',
 'The Trumpeter of Krakow',
 'Fianna Fail, Partition and Northern Ireland,1926-1971',
 'Strange Attractors',
 'Natural Cures "they" Don\'t Want You to Know about',
 'King of Thieves',
 'Suki']

In [10]:
INPUT = input("Enter the Input Query ")

res1 = es.search(
    index="books",
    query={
            "bool": {
                 "must": [
                {"match": {"title": INPUT}},
                {"range": {"pageCount": {"gte": 200, "lte": 300}}},
                {"range": {"averageRating": {"gte": 2.0, "lte": 5.0}}}
            ]
                },
                
            },
                    
)

title = [x['_source']['title']  for x in res1['hits']['hits']] 
title


[]

In [9]:
search_term = input("Enter the title you want to search: ")
resp = es.search(
    index="books",
    query={
            "bool": {
                "must": {
                    "match_phrase": {
                        "title": search_term,
                    }
                },
                
            },
        },            
)
print(resp.body)
# Extract hits from the response
hits = resp['hits']['hits']

# Extract relevant information from each hit
data = [{'id': hit['_source']['id'],
         'title': hit['_source']['title'],
         'desc': hit['_source']['desc'],
         'authors': hit['_source']['authors'],
         'categories': hit['_source']['categories'],
         'averagerating': hit['_source']['averagerating'],
         'maturityrating': hit['_source']['maturityrating'],
         'publisheddate': hit['_source']['publisheddate'],
         'pagecount': hit['_source']['pagecount']} for hit in hits]

# Create a DataFrame
df_result = pd.DataFrame(data)

try:
    existing_df = pd.read_csv('result.csv')
except FileNotFoundError:
    existing_df = pd.DataFrame()

    # Append new results to the existing DataFrame
updated_df = pd.concat([existing_df, df_result], ignore_index=True)

    # Save the updated DataFrame to 'result.csv' (overwrite)
updated_df.to_csv('result.csv', index=False)
print("Results appended to result.csv")

{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 71, 'relation': 'eq'}, 'max_score': 6.0506496, 'hits': [{'_index': 'books', '_id': 'TrHO0onCR-EC', '_score': 6.0506496, '_source': {'id': 'TrHO0onCR-EC', 'title': 'The End of Harry Potter?', 'desc': "The publication of Harry Potter and the Deathly Hallows, the seventh and final Harry Potter novel, is probably the most eagerly anticipated event in the history of publishing. Even the smallest hints from author J. K. Rowling about what may happen to Harry and his friends have been major news stories. In The End of Harry Potter?, David Langford—Potter fan and award-winning writer—delves into the many mysteries which remain unsolved. Is Albus Dumbledore really dead? Whose side is Severus Snape really on? What are the remaining horcruxes, where He Who Shall Not Be Named has stashed his soul? Does Harry bear a part of the Dark Lord's soul in his scar, and is this wh